In [38]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import preprocessing
from IPython.display import display

In [16]:
mciModel = pd.read_csv('full_db.csv')

In [17]:
mciModel2014 = mciModel[mciModel['OCC_YEAR']>= 2014]
mciModel2014.shape

(321843, 35)

In [18]:
# Remove all the roes contains NSA (No Specific Area) string values 
mciModel2014.drop(mciModel2014[mciModel2014.HOOD_140 == 'NSA'].index, inplace=True)

C:\Users\WithA\AppData\Local\Temp\ipykernel_5432\3138154694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mciModel2014.drop(mciModel2014[mciModel2014.HOOD_140 == 'NSA'].index, inplace=True)


In [19]:
# Label encode for categorical columns
label_encoder = preprocessing.LabelEncoder()
mciModel2014['PREMISES_TYPE_Code']= label_encoder.fit_transform(mciModel2014['PREMISES_TYPE'])  
mciModel2014['MCI_CATEGORY_Code']= label_encoder.fit_transform(mciModel2014['MCI_CATEGORY']) 

C:\Users\WithA\AppData\Local\Temp\ipykernel_5432\87000198.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mciModel2014['PREMISES_TYPE_Code']= label_encoder.fit_transform(mciModel2014['PREMISES_TYPE'])
C:\Users\WithA\AppData\Local\Temp\ipykernel_5432\87000198.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mciModel2014['MCI_CATEGORY_Code']= label_encoder.fit_transform(mciModel2014['MCI_CATEGORY'])


In [20]:
display(mciModel2014['PREMISES_TYPE_Code'].unique())
display(mciModel2014['MCI_CATEGORY_Code'].unique())

array([2, 0, 7, 1, 9, 4, 6, 5, 8, 3])

array([2, 0, 3, 1, 4])

In [26]:
# Separate features from target data
X = mciModel2014[['OCC_DOY','OCC_HOUR','PREMISES_TYPE_Code','lowinc_percent','Female Unemployment Ratio','HOOD_140']]
y = mciModel2014['MCI_CATEGORY_Code']

In [27]:
# Scale the features
MinMaxScaler = preprocessing.MinMaxScaler()
X_minmax = MinMaxScaler.fit_transform(X)

In [28]:
X = pd.DataFrame(X_minmax,columns=['OCC_DOY','OCC_HOUR','PREMISES_TYPE_Code','lowinc_percent','Female Unemployment Ratio','HOOD_140'])
X.head()

,OCC_DOY,OCC_HOUR,PREMISES_TYPE_Code,lowinc_percent,Female Unemployment Ratio,HOOD_140
0,0.0,0.090909,0.222222,0.287500,0.192661,0.496403
1,0.0,0.136364,0.000000,0.782143,0.770642,0.525180
2,0.0,0.136364,0.777778,0.100000,0.146789,0.467626
3,0.0,0.045455,0.111111,0.376786,0.165138,0.546763
4,0.0,0.000000,0.222222,0.492857,0.440367,0.899281


In [29]:
y =list(y)
mn = sm.MNLogit(y,sm.add_constant(X))


In [30]:
model = mn.fit()
print_model = model.summary()
print(print_model)

Optimization terminated successfully.
         Current function value: 1.166787
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                      y   No. Observations:               316452
Model:                        MNLogit   Df Residuals:                   316424
Method:                           MLE   Df Model:                           24
Date:                Sat, 15 Jul 2023   Pseudo R-squ.:                 0.07936
Time:                        22:32:01   Log-Likelihood:            -3.6923e+05
converged:                       True   LL-Null:                   -4.0106e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                      y=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.1434      0.024    -87.946      0.000      -2.

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [32]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)
ypred=knn_clf.predict(X_test)

In [33]:
result = confusion_matrix(y_test, ypred)
print("Confusion Matrix:")
print(result)

Confusion Matrix:
[[27046  2224  3163  1119    98]
 [ 3829  4209   696   332    52]
 [ 6276   842  5096   237    39]
 [ 3799   613   512  1075    21]
 [ 1184   348   368    87    26]]


In [34]:
result1 = classification_report(y_test, ypred)
print("Classification Report:",)
print (result1)

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.80      0.71     33650
           1       0.51      0.46      0.49      9118
           2       0.52      0.41      0.46     12490
           3       0.38      0.18      0.24      6020
           4       0.11      0.01      0.02      2013

    accuracy                           0.59     63291
   macro avg       0.43      0.37      0.38     63291
weighted avg       0.56      0.59      0.56     63291



In [35]:
result2 = accuracy_score(y_test,ypred)
print("Accuracy:",result2)

Accuracy: 0.591742901834384


In [39]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=12,n_estimators=100, oob_score=True)
classifier_rf.fit(X_train, y_train)
classifier_rf.oob_score_


0.621600483486793

In [40]:
dtree = DecisionTreeClassifier(criterion="entropy", max_depth=12)
dtree = dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6107661436855161
